In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from random import randint, choice, uniform
import string
import time

# Define colors and image dimensions
image_width = 250
image_height = 70
font_size = 50
characters = [5, 5]

# Define the list of allowed characters
allowed_characters = "023456789ABCDEFHJKLMNPQRTUVWXYabdefghikmnqrty"

def random_color():
    return (randint(0, 255), randint(0, 255), randint(0, 255))

def get_text():
    out_string = ""
    for i in range(randint(characters[0], characters[1])):
        out_string += choice(allowed_characters)
    return out_string

def draw_pixel(draw, x, y, thickness):
    color = random_color()
    if thickness > 1:
        draw.line([(x, y), (x + thickness * ([1, -1][randint(0, 1)]), y + thickness * ([1, -1][randint(0, 1)]))], fill=color, width=thickness)
    else:
        draw.line([(x, y), (x, y)], fill=color)

def add_noise(draw, amount, thickness):
    for i in range(int(amount)):
        draw_pixel(draw, randint(0, image_width), randint(0, image_height), thickness)

def add_lines(draw, amount, thickness):
    for i in range(int(amount)):
        color = random_color()
        draw.line([(randint(0, image_width), (randint(0, image_height))), (randint(0, image_width), randint(0, image_height))], fill=color, width=thickness)

def draw_characters(draw, image, text):
    fonts = [
        #Arial_Bold.ttf"
        "ComicSansMS3.ttf",
        "carbontype.ttf",
        "Kingthings_Trypewriter_2.ttf",
        "Sears_Tower.ttf",
        "TravelingTypewriter.ttf",
    ]
    wiggle_room_width = 10
    wiggle_room_height = 13
    font_size = 30
    wiggle_room_font_size = 10
    rotation_degrees = 20
    wiggle_room_rotation_percent = 0.4

    segment_width = int(image_width / len(text))  # Ensure segment_width is an integer
    next_letter_pos = segment_width / 2

    for character in text:
        wiggle_width = uniform(-wiggle_room_width, wiggle_room_width)
        wiggle_height = uniform(0, wiggle_room_height)
        wiggle_font_size = uniform(font_size - wiggle_room_font_size / 2, font_size + wiggle_room_font_size / 2)
        wiggle_rotation_degrees = rotation_degrees * uniform(-wiggle_room_rotation_percent, wiggle_room_rotation_percent)
        font = ImageFont.truetype("./fonts/" + fonts[randint(0, len(fonts) - 1)], int(wiggle_font_size))
        
        # Create RGBA image for character (with alpha channel)
        character_image = Image.new("RGBA", (segment_width, image_height), (255, 255, 255, 0))
        char_draw = ImageDraw.Draw(character_image)
        char_draw.text((segment_width / 4 + wiggle_width, wiggle_height), character, fill=random_color() + (255,), font=font)

        # Rotate with transparent background
        character_image = character_image.rotate(wiggle_rotation_degrees, expand=True, fillcolor=(255, 255, 255, 0))

        # Convert the main image to RGBA if it isn't already
        if image.mode != 'RGBA':
            image = image.convert('RGBA')

        # Paste using alpha compositing
        image.alpha_composite(character_image, (int(next_letter_pos - segment_width / 2), 0))
        next_letter_pos += segment_width

# Get user input for the number of images
num_samples = input("How many images do you want to generate? ")
text_done = []
start = time.time()

for i in range(int(num_samples)):
    # Create image with RGBA mode
    image = Image.new("RGBA", (image_width, image_height), (255, 255, 255, 255))
    draw = ImageDraw.Draw(image)

    text = get_text()
    if text not in text_done:
        text_done.append(text)
    else:
        print("Skipped")
        continue

    draw_characters(draw, image, text)

    add_noise(draw, amount=int(image_width * image_height * 0.001), thickness=0.5)
    add_noise(draw, amount=int(image_width * image_height * 0.001), thickness=1)
    add_noise(draw, amount=int(image_width * image_height * 0.001), thickness=2)
    add_lines(draw, amount=1, thickness=1)
    add_lines(draw, amount=1, thickness=2)

    # Convert to RGB before saving as JPG
    image = image.convert('RGB')
    image.save("./ImgCaptchaDiffFonts/" + text + ".jpg")

end = time.time()
print("Operation completed. Time taken: " + str(end - start) + " seconds")


How many images do you want to generate?  5000
